
# Web Scraping

El web scraping es una técnica que permite extraer datos e información de una web. 

Herramientas que podrias usar para ello:
 - Utilizar las librerías requests y Beautiful Soup de manera conjunta (es lo que veremos en este tutorial).
 - Utilizar Selenium. Esta herramienta te va a ser util para interactuar con la web y extraer informacion de esta de una manera muy rápida.
 - Utilizar el framework Scrapy. Este framework, además de hacer scraping (extraer información de una página) permite hacer crawling fácilmente (descubrir los enlaces de una web y navegar a través de ellos).    

### Que es Beautiful Soup

Beautiful Soup es una librería Python que permite extraer información de contenido en formato HTML o XML. Para usarla, es necesario especificar un parser, que es responsable de transformar un documento HTML o XML en un árbol complejo de objetos Python. Esto permite, por ejemplo, que podamos interactuar con los elementos de una página web como si estuviésemos utilizando las herramientas del desarrollador de un navegador.

A la hora de extraer información de una web, uno de los parsers más utilizado es el parser HTML de lxml. 

Para instalar la librerias, lo ideal es crear un entorno virtual e instalarlas una vez activado el entorno, para ello

<code> pip install beautifulsoup4
<code> pip install lxml

*Documentacion en https://beautiful-soup-4.readthedocs.io/en/latest/*

### Pasospara hacer web scraping

#### 1 – Identifica los elementos de la página de los que extraer la información
Las páginas web son documentos estructurados formados por una jerarquía de elementos. El primer paso para extraer información es identificar correctamente el elemento o elementos que contienen la información deseada.

Lo más fácil es abrir la página en un navegador e inspeccionar el código. Veremos un ejemplo.

**Importante**, mira las etiquetas, atributos, id, class, ya que posteriormente sera lo que se use para obtener la información

#### 2 – Descarga el contenido de la página
Usar la librería requests. El contenido de la respuesta, el que contiene la página en HTML, es lo que se pasa a Beautiful Soup para generar el árbol de elementos y poder hacer consultas al mismo.

#### 3 – Crear la «sopa»
Pasar el contenido de la respuesta a Beautiful Soap es lo que se conoce como hacer la «sopa», esto es, el árbol de objetos Python que representan al documento HTML.

Para ello, hay que crear un objeto de tipo BeautifulSoup, al cuál se le pasa el texto en formato HTML y el identificador del parser a utilizar

>import requests

>from bs4 import BeautifulSoup

>r = request.get('https://www.comillas.edu/icai')

>soup = BeautifulSoup(r.text,'lxml')

#### 4 – Busca los elementos en la «sopa» y obtén la información deseada
El último paso es hacer una búsqueda en el árbol y obtener los objetos que contienen la información y datos que necesitamos. 

### Tipos de objetos de Beautiful Soup
El objeto tipo BeautifulSoup que se crea cuando se añade la respuesta de la request, que representa al árbol de objetos Python resultante de parsear el documento HTML de entrada, será el punto de partida para navegar a través de los elementos del árbol, así como para realizar las búsquedas necesarias en el mismo.


In [3]:
import requests

from bs4 import BeautifulSoup

r = requests.get('https://www.comillas.edu/icai')

soup = BeautifulSoup(r.text,'lxml')

### Tag
Este objeto se corresponde con una etiqueta HTML o XML. Por ejemplo, dado el objeto soup, podemos acceder al objeto (tag) que representa al título de la página usando la etiqueta title

In [4]:
soup.title

<title>ICAI</title>

Dado un objeto de tipo Tag, podemos acceder a sus atributos tratando al objeto como si fuera un diccionario. Además, se puede acceder a ese diccionario por medio del atributo attrs:

In [7]:
div_class = soup.div
div_class['class']

['link', 'hide-mobile']

In [8]:
div_class.attrs

{'class': ['link', 'hide-mobile']}

### NavigableString
Un objeto de este tipo representa a la cadena de texto que hay contenida en una etiqueta. Se accede por medio de la propiedad string:

In [9]:
primer_parrafo= soup.p
texto = primer_parrafo.string
texto

'Menu'

In [10]:
type(texto)

bs4.element.NavigableString

Además, la clase NavigableString es clase padre de otras, entre las que se encuentran:

 - Comment: Esta clase representa un comentario HTML
 - Stylesheet: Esta clase representa un código CSS embebido
 - Script: Esta clase representa un código Javascript embebido

### Navegar a través de los elementos de Beautiful Soup

A continuación vamos a ver cómo podemos navegar a través de los objetos del árbol de Beautiful Soup.

Lo importante en este punto es que un objeto de tipo Tag puede contener bien otros objetos de tipo Tag, bien objetos de tipo NavegableString.

Además, usando los nombres de etiquetas se puede navegar hacia los elementos interiores de un objeto.

In [18]:
inter_element =soup.g.g
inter_element.attrs

{'id': 'MENU',
 'transform': 'translate(-52.000000, -154.000000)',
 'stroke': '#9B9B9B'}

Sin embargo, esta forma de acceder a los elementos puede que no sea la más adecuada en todas las situaciones. ¿Por qué? Porque de este modo, Beautiful Soup solo devuelve el primer elemento que se corresponda con la etiqueta.

### Hijos y descendientes de un elemento en Beautiful Soup
Hay otras formas de acceder a los hijos de un objeto, además de a través de las etiquetas:

 - **El atributo contents**: Devuelve una lista con todos los hijos de primer nivel de un objeto.
 - **Usando el generador children**: Devuelve un iterador para recorrer los hijos de primer nivel de un objeto.
 - **Por medio del generador descendants**: Este atributo devuelve un iterador que permite recorrer todos los hijos de un objeto. No importa el nivel de anidamiento.

In [29]:
iner_4g = soup.g.g.g.g

In [30]:
#contents
hijos = iner_4g.contents
type(hijos)
for child in hijos:
    if child.name:
        print(child.name)

path
path


In [33]:
#children
hijos = iner_4g.children
type(hijos)
for child in hijos:
    if child.name:
        print(child.name)

path
path


In [35]:
#descendants
hijos = iner_4g.descendants
type(hijos)
for child in hijos:
    if child.name:
        print(child.name)

path
path


### Padres de un elemento
Además de a los hijos, es posible navegar hacia arriba en el árbol accediendo a los objetos padre de un elemento. Usando las propiedades parent y parents:

 - parent referencia al objeto padre de un elemento (Tag o NavigableString).
 - parents es un generador que permite recorrer recursivamente todos los elementos padre de uno dado.

## Buscar elementos en Beautiful Soup

Beautiful Soup dispone de diferentes métodos para buscar elementos en el árbol. Sin embargo, dos de los principales son find_all() y find().

Ambos métodos trabajan de forma similar. Básicamente, buscan entre los descendientes de un objeto de tipo Tag y recuperan todos aquellos que cumplan una serie de filtros.

### Filtro por nombre de etiqueta
El filtro más básico consiste en pasar el nombre de la etiqueta a buscar como primer argumento de la función (parámetro name).

In [36]:
#buscar etiqueta link
enlaces = soup.find_all('link')

for enlace in enlaces:
    print(enlace)

<link href="https://cookiescdn.elixregtech.com/o/1/1/3/estyle_min.css" rel="stylesheet"/>
<link href="https://cookiescdn.elixregtech.com/o/1/1/3/client.css" rel="stylesheet"/>
<link href="https://fonts.googleapis.com/css?family=Source+Sans+Pro:300,400,700" rel="stylesheet"/>
<link href="/templates/comillascentros/css/libs/owl.carousel.css" rel="stylesheet" type="text/css"/>
<link href="/templates/comillascentros/css/libs/owl.theme.default.css" rel="stylesheet" type="text/css"/>
<link href="/templates/comillascentros/css/site.css" rel="stylesheet" type="text/css"/>
<link href="/templates/comillascentros/css/home.css" rel="stylesheet" type="text/css"/>
<link href="/templates/comillascentros/css/comillas.css" rel="stylesheet" type="text/css"/>
<link href="/templates/comillascentros/css/propios.css" rel="stylesheet" type="text/css"/>
<link href="/media/coalawebsociallinks/modules/sociallinks/css/cw-default.css" rel="stylesheet"/>
<link href="/media/coalawebsociallinks/components/sociallink

### Filtro por atributos
Además del nombre de la etiqueta, se puede especificar parámetros con nombre. Si estos no coinciden con los nombres de los parámetros de la función, serán tratados como atributos de la etiqueta entre los que filtrar.

In [37]:
ico_form = soup.find_all(id='ICO/FORMULARIO-GRADO')
print(ico_form)

[<g fill="none" fill-rule="evenodd" id="ICO/FORMULARIO-GRADO" stroke="none" stroke-width="1">
<g fill="#FFB81C" fill-rule="nonzero" id="change_c" transform="translate(4.000000, 16.000000)">
<path d="M13.5411417,8.77076189e-15 L0.860433071,8.77076189e-15 C0.384448819,8.77076189e-15 -4.4408921e-15,0.384448819 -4.4408921e-15,0.860433071 C-4.4408921e-15,1.33641732 0.384448819,1.72086614 0.860433071,1.72086614 L13.5411417,1.72086614 C14.017126,1.72086614 14.4015748,1.33641732 14.4015748,0.860433071 C14.4015748,0.384448819 14.017126,8.77076189e-15 13.5411417,8.77076189e-15 Z" id="Shape"></path>
<path d="M13.5411417,4 L0.860433071,4 C0.384448819,4 -4.4408921e-15,4.38444882 -4.4408921e-15,4.86043307 C-4.4408921e-15,5.33641732 0.384448819,5.72086614 0.860433071,5.72086614 L13.5411417,5.72086614 C14.017126,5.72086614 14.4015748,5.33641732 14.4015748,4.86043307 C14.4015748,4.38444882 14.017126,4 13.5411417,4 Z" id="Shape"></path>
<path d="M4.54114173,8 L0.860433071,8 C0.384448819,8 0,8.38425533 0

### Filtro por clases CSS
Otro filtro muy común que se suele aplicar es identificar un elemento a partir de sus clases de CSS.

In [39]:
link_divs = soup.find_all('div',class_='content')
print(link_divs)

[<div class="content">
<p>
                        Tour virtual                    </p>
</div>, <div class="content">
<p>
                    SOLICITAR ADMISIÓN                </p>
</div>, <div class="content form">
<p>
                        Descargar folleto                    </p>
<?php //Pedro López 20-7-2022 - Mensaje de Vacaciones?>
<!--<hr>
                    <p style="font: normal 13px/15px 'Source Sans Pro';text-transform: none;text-align: left;">La Oficina de Nuevos Alumnos permanecerá cerrada del <strong>22 de julio al 24 de agosto</strong>.
                        <br>Le responderemos lo antes posible a la vuelta.
                        <br>Para incidencias técnicas urgentes, por favor, contacte con: incidencias.stic@comillas.edu
                        <br>Gracias.
                    </p>
                    <hr>-->
<form action="https://www.comillas.edu/solicitainformacion/enviarformularioleads.php" data-parsley-validate="" id="formularioleads" method="post" role="for

### Limitando las búsquedas
find_all() devuelve una lista de objetos. Internamente, el método tiene que recorrer todos los hijos del objeto invocador en busca de aquellos que cumplan los diferentes filtros. En ocasiones, si el árbol (o documento) es muy grande, esta función puede tardar un poco. Se puede optimizar con los siguientes parámetros:

 - limit: Indica el número de resultados a devolver por la función. Una vez encontrados, la función deja de buscar.
 - recursive: Indica si la función debe buscar entre todos los elementos hijos de un objeto o solo aquellos de primer nivel. Por defecto es True.